<a href="https://colab.research.google.com/github/LJ1201/MSDS-490-Bank_Campaign/blob/main/BankMkting_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import packages

In [5]:
# !pip install gower

In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import metrics
# Confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Clustering algorithm
from sklearn.cluster import AgglomerativeClustering

# Rand Index
from sklearn.metrics.cluster import rand_score

# Encode labels
from sklearn import preprocessing


import gower

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Import dataset

Data Source: http://archive.ics.uci.edu/ml/datasets/Bank+Marketing#

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

Data Set Information:

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

There are four datasets:
1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]
2) bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
3) bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs).
4) bank.csv with 10% of the examples and 17 inputs, randomly selected from 3 (older version of this dataset with less inputs).
The smallest datasets are provided to test more computationally demanding machine learning algorithms (e.g., SVM).

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).


##Attribute Information:

Input variables:

#Bank client data:

1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')



Related with the last contact of the current campaign:

8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#Other attributes:

12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)


17 - cons.price.idx: consumer price index - monthly indicator (numeric)


18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)


19 - euribor3m: euribor 3 month rate - daily indicator (numeric)


20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):


21 - y - has the client subscribed a term deposit? (binary: 'yes','no')




#Relevant Papers:

S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimaraes, Portugal, October, 2011. EUROSIS. [bank.zip]



In [7]:
url = 'https://raw.githubusercontent.com/LJ1201/MSDS-490-Bank_Campaign/main/bank-additional-full.csv'
data = pd.read_csv(url, sep=';')

In [8]:
data_full = data.copy()
data_full = data.drop(['duration'],axis = 1)

In [9]:
np.random.seed(42)

In [10]:
data_full_yes = data_full[data_full['y'] == 'yes']
data_full_no = data_full[data_full['y'] == 'no']

In [11]:
data_full_no = data_full_no.sample(frac = 0.3)

In [12]:
data_full_sample = pd.concat([data_full_yes, data_full_no], sort=False)

In [13]:
data_full_sample = data_full_sample.reset_index()
data_full_sample = data_full_sample.drop(['index'], axis = 1)

In [14]:
# data_full_yes = data_full_yes.reset_index()

In [15]:
# data_full_yes= data_full_yes.drop(['index'], axis = 1)

In [16]:
# target = data_full_yes[['y']]
# data_no_target = data_full_yes.drop(['y'],axis=1)
# data_no_target.head()

In [17]:
target = data_full_sample[['y']]
data_no_target = data_full_sample.drop(['y'],axis=1)
data_no_target.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,41,blue-collar,divorced,basic.4y,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,49,entrepreneur,married,university.degree,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,49,technician,married,basic.9y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,41,technician,married,professional.course,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,45,blue-collar,married,basic.9y,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


In [18]:
data_no_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15604 entries, 0 to 15603
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             15604 non-null  int64  
 1   job             15604 non-null  object 
 2   marital         15604 non-null  object 
 3   education       15604 non-null  object 
 4   default         15604 non-null  object 
 5   housing         15604 non-null  object 
 6   loan            15604 non-null  object 
 7   contact         15604 non-null  object 
 8   month           15604 non-null  object 
 9   day_of_week     15604 non-null  object 
 10  campaign        15604 non-null  int64  
 11  pdays           15604 non-null  int64  
 12  previous        15604 non-null  int64  
 13  poutcome        15604 non-null  object 
 14  emp.var.rate    15604 non-null  float64
 15  cons.price.idx  15604 non-null  float64
 16  cons.conf.idx   15604 non-null  float64
 17  euribor3m       15604 non-null 

In [19]:
data_no_target.nunique()

age                75
job                12
marital             4
education           8
default             3
housing             3
loan                3
contact             2
month              10
day_of_week         5
campaign           35
pdays              26
previous            7
poutcome            3
emp.var.rate       10
cons.price.idx     26
cons.conf.idx      26
euribor3m         302
nr.employed        11
dtype: int64

In [20]:
data_no_target_trans = data_no_target.copy()

##Numeric Variables Transform

Age

In [21]:
data_no_target_trans['age'] = data_no_target_trans['age']//10

In [22]:
data_no_target_trans.nunique()

age                 9
job                12
marital             4
education           8
default             3
housing             3
loan                3
contact             2
month              10
day_of_week         5
campaign           35
pdays              26
previous            7
poutcome            3
emp.var.rate       10
cons.price.idx     26
cons.conf.idx      26
euribor3m         302
nr.employed        11
dtype: int64

In [23]:
data_no_target_trans.head().T

,0,1,2,3,4
age,4,4,4,4,4
job,blue-collar,entrepreneur,technician,technician,blue-collar
marital,divorced,married,married,married,married
education,basic.4y,university.degree,basic.9y,professional.course,basic.9y
default,unknown,unknown,no,unknown,unknown
housing,yes,yes,no,yes,yes
loan,no,no,no,no,no
contact,telephone,telephone,telephone,telephone,telephone
month,may,may,may,may,may
day_of_week,mon,mon,mon,mon,mon


Campaign

In [24]:
for i in range(0, len(data_no_target_trans)):
  if data_no_target_trans['campaign'][i] > 10:
    data_no_target_trans['campaign'][i] = 11

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Pdays into Pweeks

In [25]:
data_no_target_trans['pweeks'] =data_no_target_trans['pdays']

In [26]:
for i in range(0, len(data_no_target_trans)):
  if data_no_target_trans['pweeks'][i] <=28:
    data_no_target_trans['pweeks'][i] = data_no_target_trans['pweeks'][i] // 7
  elif data_no_target_trans['pweeks'][i] >28:
    data_no_target_trans['pweeks'][i] = 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
data_no_target_trans = data_no_target_trans.drop(['pdays'], axis = 1)

euribor3m - euribor 3 month rate

In [28]:
for i in range(0,len(data_no_target_trans)):
  if (data_no_target_trans['euribor3m'][i] >= 0) & (data_no_target_trans['euribor3m'][i] < 1) :
    data_no_target_trans['euribor3m'][i] ='0 - 1'
  elif (data_no_target_trans['euribor3m'][i] > 1 ) and (data_no_target_trans['euribor3m'][i] < 2) :
    data_no_target_trans['euribor3m'][i] ='1 - 2'
  elif (data_no_target_trans['euribor3m'][i] >= 2) and (data_no_target_trans['euribor3m'][i] < 3) :
    data_no_target_trans['euribor3m'][i] ='2 - 3'
  elif (data_no_target_trans['euribor3m'][i] >= 3) and (data_no_target_trans['euribor3m'][i] < 4) :
    data_no_target_trans['euribor3m'][i] ='3 - 4'
  elif (data_no_target_trans['euribor3m'][i] >= 4) :
    data_no_target_trans['euribor3m'][i] ='> 5'



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


cons.price.idx

In [29]:
for i in range(0,len(data_no_target_trans)):
  if (data_no_target_trans['cons.price.idx'][i] >= 92) & (data_no_target_trans['cons.price.idx'][i] < 93) :
    data_no_target_trans['cons.price.idx'][i] = '92 - 93'
  elif (data_no_target_trans['cons.price.idx'][i] >= 93 ) and (data_no_target_trans['cons.price.idx'][i] < 94) :
    data_no_target_trans['cons.price.idx'][i] ='93 - 94'
  elif (data_no_target_trans['cons.price.idx'][i] >= 94) :
    data_no_target_trans['cons.price.idx'][i] ='over 94'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


cons.conf.idx

In [30]:
for i in range(0,len(data_no_target_trans)):
  if (data_no_target_trans['cons.conf.idx'][i] >= 40)  :
    data_no_target_trans['cons.conf.idx'][i] = 'over 40'
  elif (data_no_target_trans['cons.conf.idx'][i] <40 ) :
    data_no_target_trans['cons.conf.idx'][i] ='less than 40'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [31]:
data_no_target_trans.nunique()

age                9
job               12
marital            4
education          8
default            3
housing            3
loan               3
contact            2
month             10
day_of_week        5
campaign          11
previous           7
poutcome           3
emp.var.rate      10
cons.price.idx     3
cons.conf.idx      1
euribor3m          5
nr.employed       11
pweeks             5
dtype: int64

In [32]:
data_no_target_trans_test = data_no_target_trans.copy()

In [33]:
data_no_target_trans_test.shape

(15604, 19)

In [34]:
distance_matrix = gower.gower_matrix(data_no_target_trans_test)

distance_matrix

array([[0.        , 0.15789473, 0.2631579 , ..., 0.5391201 , 0.39225078,
        0.34356657],
       [0.15789473, 0.        , 0.21052632, ..., 0.5391201 , 0.39225078,
        0.39619812],
       [0.2631579 , 0.21052632, 0.        , ..., 0.4864885 , 0.23435603,
        0.23830341],
       ...,
       [0.5391201 , 0.5391201 , 0.4864885 , ..., 0.        , 0.49716032,
        0.39584452],
       [0.39225078, 0.39225078, 0.23435603, ..., 0.49716032, 0.        ,
        0.22763158],
       [0.34356657, 0.39619812, 0.23830341, ..., 0.39584452, 0.22763158,
        0.        ]], dtype=float32)

In [115]:
encoder = preprocessing.LabelEncoder()

encoded_target = target.apply(encoder.fit_transform)

print(f'in this encoding, {encoded_target.iloc[0].values} represents {target.iloc[0].values}')

labels = pd.DataFrame()
labels['target'] = encoded_target.values.reshape(1, -1).tolist()[0]

in this encoding, [1] represents ['yes']


In [116]:
model_complete = AgglomerativeClustering(n_clusters=2, linkage='complete', affinity='precomputed')
clusters_complete = model_complete.fit_predict(distance_matrix)

In [131]:
labels['complete-predictions'] = clusters_complete

In [132]:
cri = rand_score(encoded_target.values.reshape(1, -1)[0], clusters_complete)
print(f'Rand Index: {cri}')

Rand Index: 0.5952081284100924


In [133]:
labels[['complete-predictions']].value_counts().plot.pie(autopct='%1.0f%%', pctdistance=0.7, labeldistance=1.1)

In [134]:
labels.value_counts(["target", "complete-predictions"])

target  complete-predictions
0       1                       7905
1       0                       3302
0       0                       3059
1       1                       1338
dtype: int64

The prediction seems to predict the class the other way around so we need to swap the classes.

In [135]:
labels['aligned-clusters'] = labels['complete-predictions'].apply(lambda x: int(not x))

In [136]:
labels

,target,complete-predictions,aligned-clusters
0,1,1,0
1,1,1,0
2,1,1,0
3,1,1,0
4,1,1,0
...,...,...,...
15599,0,1,0
15600,0,1,0
15601,0,0,1
15602,0,1,0


In [137]:
labels.value_counts(["target", 'aligned-clusters'])

target  aligned-clusters
0       0                   7905
1       1                   3302
0       1                   3059
1       0                   1338
dtype: int64

Text(0.5, 15.0, 'Predicted Labels')

In [138]:
cf_matrix = confusion_matrix(encoded_target.values.reshape(1, -1)[0], labels[["aligned-clusters"]].values.reshape(1, -1)[0])
cf_labels = ['True Neg','False Pos','False Neg','True Pos']
cf_labels = np.asarray(cf_labels).reshape(2,2)
fig, ax = plt.subplots(1, 1)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=cf_labels, fmt='', cmap='Blues')
ax.set_ylabel('Target Labels')    
ax.set_xlabel('Predicted Labels')

In [139]:
cf_matrix

array([[7905, 3059],
       [1338, 3302]])

In [140]:
True_neg = cf_matrix[0,0]
False_pos = cf_matrix[0,1]
True_pos = cf_matrix[1,1]
False_neg = cf_matrix[1,0]

accuracy = (True_neg + True_pos)/(True_neg + False_neg + True_pos + False_pos)
recall = (True_pos)/(False_neg+True_pos)
precision = (True_pos)/(False_pos + True_pos)
F1_score = 2 * ((precision*recall)/(precision+recall))

In [141]:
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1_score: {F1_score}')

Accuracy: 0.7182132786465009
Recall: 0.7116379310344828
Precision: 0.5191007703191323
F1_score: 0.6003090628124716
